In [2]:
from utils import *
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import torch

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
model_id = "Qwen/Qwen2.5-7B-Instruct"

In [ ]:
# Uncomment the line below if running in your own environment - the deployment is already setup for you here
create_deployment()

In [3]:
def wordle_reward(guess: str, secret_word: str) -> int:
    if guess.upper() == secret_word.upper():
        return 1   # correct guess
    else:
        return 0   # incorrect guess

In [4]:
secret_word = "POUND"

past_guesses = [
    GuessWithFeedback.from_secret(guess="CRANE", secret=secret_word),
    GuessWithFeedback.from_secret(guess="BLOND", secret=secret_word),
    GuessWithFeedback.from_secret(guess="FOUND", secret=secret_word),
]
past_guesses

[CRANE → Feedback: C(x) R(x) A(x) N(✓) E(x),
 BLOND → Feedback: B(x) L(x) O(-) N(✓) D(✓),
 FOUND → Feedback: F(x) O(✓) U(✓) N(✓) D(✓)]

In [ ]:
response = generate(get_messages(past_guesses))[0]
guess = extract_guess(response)
reward = wordle_reward(guess, secret_word)

print(f"Guessed Word: {guess} -> Reward: {reward}")

In [ ]:
def compute_advantages(rewards: list):
    rewards = np.array(rewards)
    
    # Compute the mean and standard deviation of the rewards
    mean_reward = np.mean(rewards)
    std_reward = np.std(rewards)

    # Avoid division by zero in case of zero variance (typically happens when all rewards are 0)
    # Note: In the GRPO implementation, we add 1e-4 to the std_reward to avoid division by zero
    if std_reward == 0:
        return [0] * len(rewards)

    # Divide by stddev of rewards to normalize range to 0
    advantages = (rewards - mean_reward) / std_reward
    return advantages.tolist()

In [ ]:
rewards = [0.0, 0.2, 0.4, 0.5, 0.5, 0.6, 0.8, 1.0]
compute_advantages(rewards)

In [ ]:
def render_guess_table(response, reward_fn):
    guesses = [extract_guess(guess) for guess in response]
    rewards = [reward_fn(guess, secret_word) for guess in guesses]
    print_guesses_table(guesses, rewards)

In [ ]:
print(f"Secret: {secret_word}")
response = generate(get_messages(past_guesses), num_guesses=8)
render_guess_table(response, wordle_reward)

In [ ]:
def wordle_reward_partial_credit(guess: str, secret_word: str) -> float:
    if len(guess) != len(secret_word):
        # no reward for having the wrong number of letters
        return 0.0
    
    valid_letters = set(secret_word)
    reward = 0.0
    for letter, secret_letter in zip(guess, secret_word):
        if letter == secret_letter:
            # right letter, right location
            reward += 0.2
        elif letter in valid_letters:
            # right letter, wrong location
            reward += 0.1
        else:
            # no reward
            pass
    return reward

In [ ]:
print(f"Secret: {secret_word}")
response = generate(get_messages(past_guesses), num_guesses=8, temperature=0)
render_guess_table(response, wordle_reward_partial_credit)

In [ ]:
print(f"Secret: {secret_word}")
response = generate(get_messages(past_guesses), num_guesses=8, temperature=1.3)
render_guess_table(response, wordle_reward_partial_credit)

In [ ]:
print(f"Secret: {secret_word}")
response = generate(get_messages(past_guesses), num_guesses=8, temperature=0.7)
render_guess_table(response, wordle_reward_partial_credit)